In [11]:
import pandas as pd
import json
import os
import re

In [12]:
!pwd

/teamspace/studios/this_studio


In [13]:
catalog: pd.DataFrame = pd.read_csv(
    filepath_or_buffer="nasa_space_apps_2024_seismic_detection_data/data/lunar/training/catalogs/apollo12_catalog_GradeA_final.csv"
)
catalog

,filename,time_abs(%Y-%m-%dT%H:%M:%S.%f),time_rel(sec),evid,mq_type
0,xa.s12.00.mhz.1970-01-19HR00_evid00002,1970-01-19T20:25:00.000000,73500.0,evid00002,impact_mq
1,xa.s12.00.mhz.1970-03-25HR00_evid00003,1970-03-25T03:32:00.000000,12720.0,evid00003,impact_mq
2,xa.s12.00.mhz.1970-03-26HR00_evid00004,1970-03-26T20:17:00.000000,73020.0,evid00004,impact_mq
3,xa.s12.00.mhz.1970-04-25HR00_evid00006,1970-04-25T01:14:00.000000,4440.0,evid00006,impact_mq
4,xa.s12.00.mhz.1970-04-26HR00_evid00007,1970-04-26T14:29:00.000000,52140.0,evid00007,deep_mq
...,...,...,...,...,...
71,xa.s12.00.mhz.1974-10-14HR00_evid00156,1974-10-14T17:43:00.000000,63780.0,evid00156,impact_mq
72,xa.s12.00.mhz.1975-04-12HR00_evid00191,1975-04-12T18:15:00.000000,65700.0,evid00191,impact_mq
73,xa.s12.00.mhz.1975-05-04HR00_evid00192,1975-05-04T10:05:00.000000,36300.0,evid00192,impact_mq
74,xa.s12.00.mhz.1975-06-24HR00_evid00196,1975-06-24T16:03:00.000000,57780.0,evid00196,impact_mq


In [14]:
json_file: pd.DataFrame = pd.DataFrame(columns=["velocity", "time_rel"])

In [15]:
json_file

,velocity,time_rel


In [16]:
catalog["time_rel(sec)"][0]

73500.0

# Training

In [17]:
training_dir = (
    "nasa_space_apps_2024_seismic_detection_data/data/lunar/training/data/S12_GradeA"
)

In [18]:
for i in range(len(catalog)):
    try:
        eventID = re.search('evid(\d+)', catalog['filename'][i]).group(1)
        velocity_data = pd.read_csv(
            filepath_or_buffer=f"{training_dir}/{catalog['filename'][i]}.csv"
        )["velocity(m/s)"].to_list()
        dictionary = {
            "time_rel": catalog["time_rel(sec)"][i],
            "velocity": velocity_data,
        }
        with open(
            file=f"data/lunar_train_12_a_{eventID}.json",
            mode="w",
            encoding="utf-8",
        ) as f:
            f.write(json.dumps(obj=dictionary, indent=4))
    except Exception as e:
        print(catalog["filename"][i] + " failed " + str(object=e))

xa.s12.00.mhz.1971-04-13HR00_evid00029 failed [Errno 2] No such file or directory: 'nasa_space_apps_2024_seismic_detection_data/data/lunar/training/data/S12_GradeA/xa.s12.00.mhz.1971-04-13HR00_evid00029.csv'


# Testing

In [ ]:
testing_dir = (
    "nasa_space_apps_2024_seismic_detection_data/data/lunar/test/data/"
)
testing_dirs = [testing_dir+d for d in os.listdir(testing_dir)]

SyntaxError: invalid syntax. Perhaps you forgot a comma? (2071811122.py, line 4)

In [ ]:
testing_dirs

['nasa_space_apps_2024_seismic_detection_data/data/lunar/test/data/S12_GradeB',
 'nasa_space_apps_2024_seismic_detection_data/data/lunar/test/data/S15_GradeA',
 'nasa_space_apps_2024_seismic_detection_data/data/lunar/test/data/S15_GradeB',
 'nasa_space_apps_2024_seismic_detection_data/data/lunar/test/data/S16_GradeA',
 'nasa_space_apps_2024_seismic_detection_data/data/lunar/test/data/S16_GradeB']

In [ ]:
for j in range(len(testing_dirs)):
    for i in range(len(os.listdir(testing_dirs[j]))):
        try:
            filename = f"{testing_dirs[j]}/{os.listdir(testing_dirs[j])[i]}"
            s = re.search('s(\d+)', filename).group(1)
            grade = re.search('Grade([A-Z])', filename).group(1).lower()
            eventID = re.search('evid(\d+)', filename).group(1)
            if filename.endswith('.csv') is not True:
                continue
            # print(filename)
            # print(f"{s} {grade}")
            # print(eventID)
            velocity_data = pd.read_csv(
                filepath_or_buffer=filename
            )["velocity(m/s)"].to_list()
            time_rel = pd.read_csv(
                filepath_or_buffer=filename
            )["time_rel(sec)"].to_list()
            dictionary = {
                "time_rel": time_rel,
                "velocity": velocity_data,
            }
            with open(
                file=f"data/lunar_test_{s}_{grade}_{eventID}.json",
                mode="w",
                encoding="utf-8",
            ) as f:
                f.write(json.dumps(obj=dictionary, indent=4))
        except Exception as e:
            print(filename + " failed " + str(object=e))